# Model Saving (Checkpoint.h5 > [TFModel, Token] > TFLite)

### Model Name Config

In [ ]:
#format: save_type-train_grp_num-model_name-acc-hp_hp_hp-ver

In [1]:
load_checkpoint_name = "XLM-RoBERTa__lr5e-05_ep150_bs128.h5"

In [2]:
MODEL_NAME = 'xlm-roberta-base'
saved_model_name = "first-xlm-r_94acc_lr5e-05_ep150_bs128_v3"

In [ ]:
MODEL_NAME = 'bert-base-multilingual-cased'
saved_model_name = "mbert_80pacc_try_model_v1"

In [ ]:
MODEL_NAME = 'bert-base-uncased'
saved_model_name = "bert-base_best_model_v1"

### Load Model

In [3]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

model.load_weights('checkpoints/' + load_checkpoint_name)


2025-04-20 01:05:30.515333: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 01:05:30.899111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-20 01:05:30.899202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-20 01:05:30.974971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 01:05:31.105802: I tensorflow/core/platform/cpu_feature_guar

### Save Model Type

In [ ]:
# Hugging Face Transformer Format
model.save_pretrained("saved_model/"+saved_model_name, save_format="tf")

In [ ]:
# # Keras Format
model.save("saved_model/"+saved_model_name)

#### This type below is working

In [4]:
# Standard Tensorflow Format
import tensorflow as tf
tf.saved_model.save(model, "saved_model/"+saved_model_name)

INFO:tensorflow:Assets written to: saved_model/first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/assets


INFO:tensorflow:Assets written to: saved_model/first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/assets


# Tokenizer Saving

In [5]:
tokenizer_name = "xlm-roberta-base"
# tokenizer_saved_name = "tkn_for_" + "xmlr-_fix_v1"
tokenizer_saved_name = "tkn_for_" + saved_model_name

In [6]:
from transformers import AutoTokenizer

# Assume you already have a tokenizer (for instance, fine-tuned or pretrained)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Save the tokenizer to a local directory
tokenizer.save_pretrained("saved_tokenizer/" + tokenizer_saved_name)

('saved_tokenizer/tkn_for_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/tokenizer_config.json',
 'saved_tokenizer/tkn_for_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/special_tokens_map.json',
 'saved_tokenizer/tkn_for_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/sentencepiece.bpe.model',
 'saved_tokenizer/tkn_for_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/added_tokens.json',
 'saved_tokenizer/tkn_for_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3/tokenizer.json')

# TF Model to TFLite Conversion

In [7]:
import tensorflow as tf

In [8]:
# model_input_name = "new_model_xmlr-pd-best"
model_input_name = saved_model_name

In [9]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/' + model_input_name)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter._experimental_preserve_all_tensors = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

2025-04-20 01:08:53.555439: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-04-20 01:08:53.555522: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-04-20 01:08:53.561208: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: saved_model/first-xlm-r_94acc_lr5e-05_ep150_bs128_v3
2025-04-20 01:08:53.590751: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-04-20 01:08:53.590850: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: saved_model/first-xlm-r_94acc_lr5e-05_ep150_bs128_v3
2025-04-20 01:08:53.670853: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-04-20 01:08:53.687946: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-04-20 01:08:55.079354: I tensorflow/cc/saved_model/loader.cc:217] Running 

In [12]:
with open("tflite_models/" + "lite_scamba_" + model_input_name + ".tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model converted and saved as " + "lite_scamba_" + model_input_name + ".tflite")

✅ Model converted and saved as lite_scamba_first-xlm-r_94acc_lr5e-05_ep150_bs128_v3.tflite
